In [1]:
import pandas as pd
import random
import math

Pobranie danych

In [2]:
lirabary = pd.read_csv('books.csv', on_bad_lines='skip')


lirabary.head()


,bookID,title,authors,publisher,Genres,target_groups,tags,num_pages,language_code,country_of_origin,publication_date,average_rating,ratings_count,month_rentals
0,0,Harry Potter and the Half-Blood Prince (Harry ...,"['J.K. Rowling', 'Mary GrandPré']",Scholastic Inc.,"['travel', 'magical_realism']",single_mothers,"['mafia', 'egyptian_mythology', 'angels', 'sup...",652,eng,France,2006,4.57,2095690,161206
1,1,Harry Potter and the Order of the Phoenix (Har...,"['J.K. Rowling', 'Mary GrandPré']",Scholastic Inc.,"['parenting_and_families', 'short_story']",young_adults,"['gore', 'egyptian_mythology', 'love_triangle'...",870,eng,Poland,2004,4.49,2153167,165628
2,2,Harry Potter and the Chamber of Secrets (Harry...,['J.K. Rowling'],Scholastic,"['parenting_and_families', 'contemporary_ficti...",boys,"['time_travel', 'gambling']",352,eng,Mexico,2003,4.42,6333,487
3,3,Harry Potter and the Prisoner of Azkaban (Harr...,"['J.K. Rowling', 'Mary GrandPré']",Scholastic Inc.,['young_adult'],introverts,"['time_travel', 'violence', 'slice_of_life']",435,eng,Korea,2004,4.56,2339585,179968
4,4,Harry Potter Boxed Set Books 1-5 (Harry Potte...,"['J.K. Rowling', 'Mary GrandPré']",Scholastic,"['horror', 'action_adventure']",engaged_people,"['egyptian_mythology', 'isekai', 'gods', 'egyp...",2690,eng,Russia,2004,4.78,41428,3186


In [3]:
def Make_List_From_txt(title: str):
    genres = open('additional_data_base_info/' + title + '.txt','r')
    genres_list =[]
    for line in genres:
        genres_list.append(line.strip())
    return genres_list

def ToHamming(all_possibilities, vector):
    result = []
    for x in all_possibilities:
        if(x in vector):
            result.append(1)
        else:
            result.append(0)
    return result

def Haming_Similaryty(vector1:list, vector2:list):
    wynik = 0
    if(len(vector1) != len(vector2)):
        return -1
    
    for i in range(0, len(vector1)-1):
        if(vector1[i] == vector2[i]):
            wynik = wynik +1
    return wynik


Odległość haminga

In [4]:
import numpy as np

genres = Make_List_From_txt("genres")
tags = Make_List_From_txt("tags")

column_to_haming = {"Genres": genres, "tags": tags}

for key in column_to_haming:
    lirabary["haminged_" + key] =  lirabary[key].apply(lambda x: ToHamming(column_to_haming[key], x) )
    


Odległość cosinusowa

In [5]:
import math
from ast import literal_eval
def CosSimilarity(vector_1, book_vector, max_num_pages, max_publicate_data):
    columns_to_cos_metrice = ["publisher", "target_groups", "num_pages", "language_code", "country_of_origin", "publication_date"]
    maxs = {"num_pages": max_num_pages, "publication_date": max_publicate_data}
    squer_sum_bookVector = 0
    squer_sum_vector = 0
    vector_1__multiplay__book_vector = 0

    squer_sum_bookVector += 1
    if any( item in literal_eval(vector_1["authors"]) for item in literal_eval(book_vector["authors"]) ):
        vector_1__multiplay__book_vector += 1
        squer_sum_vector += 1
    
    for name in columns_to_cos_metrice:
        if(type(vector_1[name]) == str):
            squer_sum_bookVector += 1
            if(vector_1[name] == book_vector[name]):
                vector_1__multiplay__book_vector += 1
                squer_sum_vector += 1
        else:
            squer_sum_bookVector += math.pow(book_vector[name] / maxs[name],2)
            squer_sum_vector +=  math.pow(vector_1[name] / maxs[name],2)
            vector_1__multiplay__book_vector += (book_vector[name] * vector_1[name])/(maxs[name]*maxs[name])
    return vector_1__multiplay__book_vector/(math.sqrt(squer_sum_vector*squer_sum_bookVector))








Wyznaczenie biblioteki użytkownika

In [6]:
already_read = lirabary.sample(3)

Wyznaczenie podobieństwa księżek

In [7]:
max_date = lirabary["publication_date"].max()
max_pages = lirabary["num_pages"].max()
genres_lenght = len(lirabary["haminged_Genres"].iloc[0])
tags_lenght = len(lirabary["haminged_tags"].iloc[0])
for index, row in already_read.iterrows():
    lirabary[row["bookID"]] =  lirabary.apply(lambda x:
        CosSimilarity(x, row, max_pages,max_date) + 
        Haming_Similaryty(x["haminged_Genres"], row["haminged_Genres"])/genres_lenght +
        Haming_Similaryty(x["haminged_tags"], row["haminged_tags"])/tags_lenght , axis=1)




Suma metryk

In [8]:
bookid = already_read["bookID"]
lirabary["absolute_metric"] = lirabary[bookid].sum(axis=1)

In [9]:
already_read.iloc[0]

bookID                                                            9049
title                                       The Essential Frankenstein
authors              ['Mary Wollstonecraft Shelley', 'Leonard Wolf'...
publisher                                                        Plume
Genres                         ['horror', 'lgbtq+', 'food_and_drinks']
target_groups                                           single_fathers
tags                 ['wolves', 'time_travel', 'male_protagonist', ...
num_pages                                                          357
language_code                                                      eng
country_of_origin                                                China
publication_date                                                  1993
average_rating                                                    3.79
ratings_count                                                       42
month_rentals                                                        3
haming

In [10]:
lirabary = lirabary.sort_values(already_read.iloc[0]["bookID"], ascending=False)
lirabary[:5]

,bookID,title,authors,publisher,Genres,target_groups,tags,num_pages,language_code,country_of_origin,publication_date,average_rating,ratings_count,month_rentals,haminged_Genres,haminged_tags,9049,10784,9594,absolute_metric
9049,9049,The Essential Frankenstein,"['Mary Wollstonecraft Shelley', 'Leonard Wolf'...",Plume,"['horror', 'lgbtq+', 'food_and_drinks']",single_fathers,"['wolves', 'time_travel', 'male_protagonist', ...",357,eng,China,1993,3.79,42,3,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",2.950751,2.382493,2.173082,7.506326
7125,7125,The Analects of Confucius: A Philosophical Tra...,"['Confucius', 'Henry Rosemont Jr.', 'Roger T. ...",Ballantine Books,"['food_and_drinks', 'thriller_and_suspense', '...",single_fathers,"['time_travel', 'male_protagonist', 'vikings']",352,eng,China,1999,3.83,180,13,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2.569443,2.471381,2.261943,7.302767
9427,9427,Their Eyes Were Watching God,['Zora Neale Hurston'],Amistad,['lgbtq+'],single_fathers,"['vikings', 'demons']",219,eng,China,2006,3.91,220309,16946,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2.556782,2.547560,2.337529,7.441872
6743,6743,No god but God: The Origins Evolution and Fut...,['Reza Aslan'],Random House Trade,"['fantasy', 'religion_and_spirituality']",single_fathers,"['vikings', 'male_protagonist', 'time_travel',...",310,eng,China,2006,4.12,16918,1301,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",2.542408,2.453943,2.298370,7.294720
9149,9149,The Life You Were Born to Live,['Dan Millman'],MJF Books,"['biography', 'historical_fiction']",single_fathers,"['fatal_disease', 'violence', 'wolves', 'time_...",464,eng,China,2000,4.11,19,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",2.542389,2.453982,2.245008,7.241378


Testowanie finalnego algorytmu

In [11]:
lirabary2 = pd.read_csv('books.csv', on_bad_lines='skip')

sample = lirabary2.sample(10)

import Recomender as ra

result = ra.Recommend(sample)

Porównanie rekomendaci dla pierwszej wylosowanej książki

In [12]:
sample.iloc[0]

bookID                                                           10185
title                                                  The White Widow
authors                                                 ['Jim Lehrer']
publisher                                                PublicAffairs
Genres               ['self_help', 'horror', 'humanities_and_social...
target_groups                                           single_fathers
tags                  ['betrayal', 'magic', 'dance', 'ancient_greece']
num_pages                                                          224
language_code                                                    en-US
country_of_origin                                                China
publication_date                                                  2000
average_rating                                                    3.04
ratings_count                                                      181
month_rentals                                                       13
Name: 

In [13]:
result.sort_values(sample.iloc[0]["bookID"], ascending=False)

,bookID,title,authors,publisher,Genres,target_groups,tags,num_pages,language_code,country_of_origin,...,2791,5713,1859,4211,2605,6008,7665,11068,7035,Sum_metrics
2787,2787,Things to Bring S#!t to Do: And Other Invento...,['Karen Rizzo'],Stewart Tabori and Chang,['horror'],elders,"['ancient_greece', 'magic']",160,eng,United_Kingdom,...,2.532160,2.487874,2.438436,2.564066,2.465555,2.370007,2.336538,2.497290,2.398800,24.918194
8081,8081,Who Are You?: 101 Ways of Seeing Yourself,['Malcolm Godwin'],Penguin Books,['self_help'],young_children,"['betrayal', 'magic']",224,eng,Italy,...,2.532094,2.487847,2.438355,2.519528,2.465487,2.398723,2.460601,2.497192,2.398720,25.025766
10893,10893,Stir-Fry,['Emma Donoghue'],Alyson Books,"['horror', 'self_help']",engaged_people,"['ancient_greece', 'archery']",240,eng,Russia,...,2.460601,2.460807,2.411302,2.536916,2.438437,2.425725,2.389102,2.470134,2.371668,24.774416
9532,9532,Philosophical Papers: Volume 1 Human Agency a...,['Charles Taylor'],Cambridge University Press,"['horror', 'self_help']",adults,"['betrayal', 'greek_mythology', 'betrayal']",304,eng,Poland,...,2.457343,2.457590,2.408028,2.406399,2.435178,2.422454,2.385821,2.466844,2.412840,24.661835
5081,5081,Henry and June: From the Unexpurgated Diary of...,['Anaïs Nin'],Houghton Mifflin Harcourt P,['horror'],children,"['gambling', 'betrayal', 'dance', 'betrayal']",274,eng,USA,...,2.462205,2.462434,2.457342,2.494061,2.357107,2.471766,2.390693,2.347502,2.373264,24.621095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3410,3410,Complete Plays 1913–1920,"[""Eugene O'Neill"", 'Travis Bogard']",Library of America,"['action_adventure', 'biography', 'new_adult']",single_fathers,"['ancient_egypt', 'angels', 'cheating', 'angel...",1107,eng,China,...,2.265528,2.382160,2.376367,2.324028,2.368845,2.390825,2.274778,2.346078,2.081939,22.999777
4657,4657,Beach Music,['Pat Conroy'],Dial Press,"['for_children', 'historical_romance', 'memoir...",single_fathers,"['cheating', 'female_protagonist', 'amnesia', ...",592,eng,China,...,2.261307,2.344048,2.348301,2.321290,2.356289,2.343518,2.326056,2.387819,2.254837,23.128600
6249,6249,"Ebert's ""Bigger"" Little Movie Glossary","['Roger Ebert', 'Ray Ebert']",Andrews McMeel Publishing,"['guide_how_to', 'historical_romance']",single_fathers,"['ghosts', 'isekai', 'fatal_disease', 'amnesia...",228,en-US,China,...,2.322480,2.640045,2.698831,2.617500,2.519214,2.497041,2.568393,2.639844,2.195186,24.874862
9261,9261,Patience Princess Catherine (Young Royals #4),['Carolyn Meyer'],HMH Books for Young Readers,"['history', 'urban_fantasy', 'graphic_novel']",single_fathers,"['first_love', 'amnesia', 'vikings', 'sex', 'g...",208,en-US,China,...,2.371754,2.590859,2.531952,2.657234,2.514492,2.492323,2.509626,2.590691,2.200007,24.630472


In [15]:
import Recomender as ra
sample["user_rating"] = sample["average_rating"]
ra.TheBest(sample, 0)

,bookID,title,authors,publisher,Genres,target_groups,tags,num_pages,language_code,country_of_origin,publication_date,average_rating,ratings_count,month_rentals,user_rating
5713,5713,Cold Fire / Hideaway / The Key to Midnight,"['Dean Koontz', 'Leigh Nichols']",Putnam Adult,"['true_crime', 'romance', 'fantasy']",boys,"['gods', 'angels', 'ghosts']",784,eng,Brazil,2000,4.18,19670,1513,4.18
7665,7665,Rebirth Volume 5,"['Kang-Woo Lee', 'Lauren Na']",TokyoPop,"['memoir_and_autobiography', 'guide_how_to', '...",elders,"['homonkulus', 'sex', 'the_chosen_one', 'sex',...",184,eng,United_Kingdom,2003,4.03,102,7,4.03
11068,11068,The Legend of Spud Murphy,"['Eoin Colfer', 'Glenn McCoy']",Disney-Hyperion,['short_story'],children,"['chinese_mythology', 'amnesia', 'mafia', 'mal...",96,eng,USA,2004,3.88,92,7,3.88
2605,2605,Half Asleep in Frog Pajamas,['Tom Robbins'],No Exit Press,"['supernatural_fiction', 'new_adult', 'histori...",teens,"['nordic_mythology', 'ancient_egypt', 'collect...",389,eng,USA,2002,3.74,18417,1416,3.74
4211,4211,I'm Telling You Stories: Jeanette Winterson an...,"['Helena Grice', 'Tim Woods']",Brill/Rodopi,['food_and_drinks'],young_adults,"['first_love', 'ghosts', 'sex', 'ancient_greece']",136,eng,Poland,1998,3.70,10,0,3.70
6008,6008,The Beatles and Philosophy: Nothing You Can Th...,"['Michael Baur', 'Steven Baur', 'James S. Spie...",Open Court,"['horror', 'biography', 'contemporary_fiction']",teens,"['fatal_disease', 'demi-gods', 'gambling', 'de...",288,eng,United_Kingdom,2006,3.63,157,12,3.63
7035,7035,Dark Water,"['Kōji Suzuki', 'Glynne Walley']",Vertical,"['historical_fiction', 'mystery', 'graphic_nov...",single_fathers,"['collective_hero', 'female_protagonist', 'ali...",279,eng,China,2006,3.61,2433,187,3.61
1859,1859,Why Do Men Fall Asleep After Sex? More Questio...,"['Mark Leyner', 'Billy Goldberg']",Three Rivers Press,"['essays', 'historical_romance', 'guide_how_to']",introverts,"['gambling', 'demi-gods', 'chinese_mythology']",263,eng,Korea,2006,3.55,1166,89,3.55
2791,2791,The Medici Giraffe and Other Tales of Exotic A...,['Marina Belozerskaya'],Little Brown and Company,"['women_s_fiction', 'history', 'biography']",single_parents,"['debut', 'magic', 'guns', 'debut']",414,eng,China,2006,3.53,224,17,3.53
10185,10185,The White Widow,['Jim Lehrer'],PublicAffairs,"['self_help', 'horror', 'humanities_and_social...",single_fathers,"['betrayal', 'magic', 'dance', 'ancient_greece']",224,en-US,China,2000,3.04,181,13,3.04
